In [ ]:
from operator import itemgetter

from langchain_core.runnables import RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama

# 具有单个参数的函数可以直接被RunnableLambda封装
def length_function(text):
    return len(text)

# 具有多个参数的函数需要先被封装成具有单个参数的函数，再传递给RunnableLambda
def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)

def multiple_length_function(_dict):
    return _multiple_length_function(_dict['text1'], _dict['text2'])

prompt = ChatPromptTemplate.from_template('what is {a} + {b}')
model = ChatOllama(model = 'qwen2.5')

chain1 = prompt | model

chain = (
    {
        'a': itemgetter('foo') | RunnableLambda(length_function),
        'b': {'text1': itemgetter('foo'), 'text2': itemgetter('bar')}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | model
)

chain.invoke({'foo': 'bar', 'bar': 'gah'})

In [1]:
from langchain_core.runnables import RunnableLambda

def add_one(x: int):
    return x + 1

runnable = RunnableLambda(add_one)

# 同步
runnable.invoke(1)   # 返回2
runnable.batch([1, 2, 3])   # 返回[2，3，4]

# 异步
# 在默认情况下，通过调用同步函数实现来支持异步调用
await runnable.ainvoke(1)
await runnable.abatch([1, 2, 3])

# 同时准备同步和异步函数实现，由RunnableLambda一同封装，按需使用
async def add_one_async(x: int):
    return x + 1

runnable = RunnableLambda(add_one, afunc = add_one_async)

runnable.invoke(1)   # 使用 add_one
await runnable.ainvoke(1)   # 使用 add_one_async

2